In [1]:
import pandas as pd
import numpy as np
import math
import random

- read data

In [2]:
data = pd.read_csv('Data set CSV.csv')
data.head()

,City,x,y
0,1,5.500000e-08,9.860000e-09
1,2,-2.887330e+01,-7.980000e-08
2,3,-7.929160e+01,-2.140330e+01
3,4,-1.465770e+01,-4.338960e+01
4,5,-6.474730e+01,2.189820e+01


- classes definitions

In [3]:
class City:
    def __init__(self, name, longitude, latitude):
        self.name = name
        self.longitude = longitude
        self.latitude = latitude

    def __str__(self):
        return f"{self.name,self.longitude,self.latitude}"

In [4]:
class Chromosome:
    def __init__(self, cities_withxy, cost, fitness):
        self.cities_withxy = cities_withxy          #each gene holds name,x,y []
        self.cost = cost
        self.fitness = fitness
    def __str__(self):
        return f"{self.cities_withxy, self.cost, self.fitness}"
    
    def __str__(self):
        return f"{self.cities_withxy, self.cost, self.fitness}"

- distance matrix

In [5]:
def calculate_distance(city1, city2):
    return math.sqrt((city1.longitude - city2.longitude)**2 + (city1.latitude - city2.latitude)**2)

def calculate_distance_matrix(cities):
    num_cities = len(cities)
    city_names = [city.name for city in cities]
    distance_matrix = {name: {other_name: 0 for other_name in city_names} for name in city_names}

    for i in range(num_cities):
        for j in range(i + 1, num_cities):
            distance = calculate_distance(cities[i], cities[j])
            distance_matrix[cities[i].name][cities[j].name] = distance
            distance_matrix[cities[j].name][cities[i].name] = distance  

    return distance_matrix 

In [7]:
def calculate_total_distance(chromosome):       #takes list of city objects
    total_distance = 0
    num_cities = len(chromosome)

    for i in range(num_cities - 1):
        city1 = chromosome[i].name
        city2 = chromosome[i + 1].name
        total_distance += distance_matrix[city1][city2]

    #add distance from the last city back to the starting city
    total_distance += distance_matrix[chromosome[-1].name][chromosome[0].name]

    return total_distance

- initial population

In [8]:
'''
Fitness Evaluation
    Fitness will be cost based
    Equation:
        Fitness = 1/ (Total cost of Tsp Trip)
        Function steps:
        a.Calculate Chromosome Cost ()
        b.Set Cost to Chromosome
        c. Calculate Chromosome Fitness(Cost)
        d.Set Fitness To Chromosome 
'''
def fitness_evaluation(chromosome_city_sequence):           #will take list of city objects
    chromosome_cost = calculate_total_distance(chromosome_city_sequence)    
    chromosome_fitness = 1 / chromosome_cost
    return chromosome_cost, chromosome_fitness

In [9]:
'''
Initial Population
Function steps:
For(Initial population count)
{
 Generate Chromosome From Permutation()
 Evaluate Chromosome Fitness()
 Add Chromosome To Population()
}
'''
def initial_population(population_size, cities):

    initial_population = []

    for _ in range(population_size):
        chromosome_sequence_with_xy = random.sample(cities, len(cities))        #list of city/gene object name,x,y  
        #print(chromosome_sequence_with_xy[0])
        chromosome_cost, chromosome_fitness = fitness_evaluation(chromosome_sequence_with_xy)

        chromosome = Chromosome(chromosome_sequence_with_xy, chromosome_cost, chromosome_fitness)
        initial_population.append(chromosome)
        
    return initial_population

- elitism

In [10]:
def sort_chromosomes(population):
    return sorted(population, key=lambda x: x.fitness, reverse=True)

In [126]:
'''
Elitism
    Function steps:
        a.Calculate Elitism Count ()
        b.Select Top Fittest Chromosomes
        c. Add Elite Chromosome To New Generation List
'''

def select_elites(population, elitism_percentage, k):
    sorted_chromosomes = sort_chromosomes(population)
    elites = sorted_chromosomes[:k]
    return elites                                                                                            

- selection

In [12]:
'''
-Selection Hint Use K-Tournament Selection
Function steps:
a.Select Random K Chromosomes ()
b.Choose the fittest()
'''
def selection(population, k_tournament):
    random_chromosomes = random.sample(population, k_tournament)
    sorted_random = sort_chromosomes(random_chromosomes)
    parent = sorted_random[0]
    return parent

- crossover

In [37]:
def partially_mapped_crossover(parent1, parent2):       #Chromo objs
    #choose two random crossover points
    crossover_point1, crossover_point2 = sorted(random.sample(range(len(parent1.cities_withxy)), 2))

    #initialize child chromosomes as copies of the parents
    child1 = parent1.cities_withxy[:]
    child2 = parent2.cities_withxy[:]

    #perform crossover within the selected points
    for i in range(crossover_point1, crossover_point2 + 1):
        #swap values between parents in the crossover section
        temp1 = child1[i]
        temp2 = child2[i]
        child1[i] = temp2
        child2[i] = temp1

    return child1, child2

In [43]:
def fix_duplicate_cities(chromosome_sequence, all_cities):
    visited_cities = set()
    new_sequence = []
    for city in chromosome_sequence:
        if city.name not in visited_cities:
            new_sequence.append(city)
            visited_cities.add(city.name)
        else:
            unvisited_cities = [c for c in all_cities if c.name not in visited_cities]
            unvisited_city = random.choice(unvisited_cities)
            new_sequence.append(unvisited_city)
            visited_cities.add(unvisited_city.name)
    return new_sequence

In [140]:
'''
Function Steps:
    a.Crossover Count =
    (population Size - No. of elite
    chromosomes) / 2
    b.for(Crossover Count)
        { parent1= Tournament selection()
        parent2= Tournament selection()
        generate random double number that ranges from 0-1
        if( generated random number < crossover
        probability){
            Apply Partial mapped crossover on
            parent1 and parent2
            Evaluate child1 and child2 Fitness
            insert child1 and child2 into new
            generation list
        }
        else {
            insert parent1 and parent2 into new
            generation list}
        }
'''

def crossover(population, k_tournament, population_size, elites, crossover_probability):
    crossover_count = (population_size - len(elites)) // 2
    new_generation = []
    for _ in range(crossover_count):
        parent1 = selection(population, k_tournament)  # select from old generation list
        parent2 = selection(population, k_tournament)
        random_number = random.random()
        if random_number < crossover_probability:
            child1, child2 = partially_mapped_crossover(parent1, parent2)  # returned city objects

            #ensure no duplicate cities in children
            child1 = fix_duplicate_cities(child1, cities)
            child2 = fix_duplicate_cities(child2, cities)

            child1_cost, child1_fitness = fitness_evaluation(child1)
            child2_cost, child2_fitness = fitness_evaluation(child2)

            child1 = Chromosome(child1, child1_cost, child1_fitness)
            child2 = Chromosome(child2, child2_cost, child2_fitness)

            return child1, child2 
        else:
            return parent1, parent2

- mutation

In [16]:
def swap_mutation(chromosome):              #Takes chromosome class, now mutate the sequence
    mutated_chromosome_sequence = chromosome.cities_withxy[:]  #sake a copy of the chromosome sequence
    
    #select two random positions
    pos1, pos2 = random.sample(range(len(mutated_chromosome_sequence)), 2)

    #swap values at the selected positions
    mutated_chromosome_sequence[pos1], mutated_chromosome_sequence[pos2] = mutated_chromosome_sequence[pos2], mutated_chromosome_sequence[pos1]

    return mutated_chromosome_sequence                  #returns list of city objects

In [141]:
'''
Function Steps:
    a.for(population size)
    {
        parent1= select random parent from new                                                          
        generation list

        generate random double number that
        ranges from 0 - 1
        if( generated random number < mutation
        probability){
            Apply swapping mutation on parent1
            Evaluate mutated parent Fitness
            replace parent1 by mutated parent into new generation list
        }
    }
'''
def mutation(population, population_size, mutation_probability, elitism_percentage):
    k = int((population_size) * elitism_percentage)
    for i in range(k, population_size):
        parent = random.choice(population)
        parent_index = population.index(parent)
        random_number = random.random()
        if random_number < mutation_probability:
            mutated_parent_sequence = swap_mutation(parent)  # returns list of city objects
            
            #ensure no city is visited more than once
            mutated_chromosome_sequence = fix_duplicate_cities(mutated_parent_sequence, cities)
            mutated_chromosome_cost, mutated_chromosome_fitness = fitness_evaluation(mutated_chromosome_sequence)
            mutated_chromosome = Chromosome(mutated_chromosome_sequence, mutated_chromosome_cost, mutated_chromosome_fitness)
            population[parent_index] = mutated_chromosome

    return population

- running the algorithm

In [18]:
cities = [City(row['City'], row['x'], row['y']) for index, row in data.iterrows()] 

In [19]:
distance_matrix = calculate_distance_matrix(cities)

In [20]:
population_size= 50
generations_count=100                                   
elitism_percentage = 0.02        #2% of population 
k = int((population_size) * elitism_percentage) #(1 chromosome)                   
crossover_probability = 0.6
mutation_probability = 0.1
k_tournament = 5                                                                                      

In [117]:
def main_loop(population_size, generations_count, elitism_percentage, crossover_probability, mutation_probability, k_tournament,k):
    
    population = initial_population(population_size, cities)
    for generation in range(generations_count):
        elites = select_elites(population, elitism_percentage, k)
        new_generation = []
        new_generation.extend(elites)

        #perform crossover to fill the rest of the population
        while len(new_generation) < population_size-k:        
            child1, child2 = crossover(population, k_tournament, population_size, elites, crossover_probability)
            new_generation.extend([child1, child2])

        #print(len(set(new_generation[0].cities_withxy)))        #test 
        #print("cv",len(set(new_generation)))                     #test 

        new_generation = mutation(new_generation, population_size, mutation_probability,elitism_percentage)
        #print("mut",len(set(new_generation)))      #test

        population = sort_chromosomes(new_generation)
        print(f"Generation {generation+1}: Best solution - Fitness: {population[0].fitness}, Cost: {population[0].cost}")
    
    #print(len(population))
    return population[0]

In [113]:
best_solution = main_loop(population_size, generations_count, elitism_percentage, crossover_probability, mutation_probability, k_tournament)

Generation 1: Best solution - Fitness: 0.002018058430996035, Cost: 495.5257908495935
Generation 2: Best solution - Fitness: 0.002020591659581517, Cost: 494.90454702119735
Generation 3: Best solution - Fitness: 0.002020591659581517, Cost: 494.90454702119735
Generation 4: Best solution - Fitness: 0.0020350249963808483, Cost: 491.3944554874908
Generation 5: Best solution - Fitness: 0.0020350249963808483, Cost: 491.3944554874908
Generation 6: Best solution - Fitness: 0.0021245232131755407, Cost: 470.6938449993646
Generation 7: Best solution - Fitness: 0.002267121390426245, Cost: 441.08798241808677
Generation 8: Best solution - Fitness: 0.002267121390426245, Cost: 441.08798241808677
Generation 9: Best solution - Fitness: 0.002496301827801369, Cost: 400.59258414306225
Generation 10: Best solution - Fitness: 0.002496301827801369, Cost: 400.59258414306225
Generation 11: Best solution - Fitness: 0.002496301827801369, Cost: 400.59258414306225
Generation 12: Best solution - Fitness: 0.00249630182

In [115]:
print("best solution found:", best_solution.cost)
print("Route: ")
for city in best_solution.cities_withxy:
    print(city.name)
print(best_solution.cities_withxy[0].name)

best solution found: 284.3810904080332
Route: 
8.0
6.0
4.0
11.0
1.0
13.0
2.0
15.0
9.0
5.0
7.0
3.0
12.0
14.0
10.0
8.0
